In [1]:
import numpy as np

In [2]:
## Here we are generating dataset


from sklearn.datasets import make_classification
from sklearn.model_selection import  train_test_split

n_x = 20
m = 10000

X, Y = make_classification(n_samples=m, n_classes=2, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(Y_train.shape)

(8000, 20)
(8000,)


In [3]:
def initialize_weights(n_x, n_h):
    '''
    Function to initalize the Weights for the Logistic regression
    
    I/P
    n_x = no. of input sfeatures
    n_h = no. of nodes in layer
    
    O/P
    W = Initial weights
    b = initial bias
    '''
    
    W_1 = np.random.randn(n_h, n_x) * 0.01
    b_1 = np.zeros((n_h, 1))
    
    W_2 = np.random.randn(1, n_h) * 0.01
    b_2 = np.zeros((1, 1))
    
    return W_1, b_1, W_2, b_2

In [4]:
def sigmoid(Z):
    '''
    Function to calculate sigmoid activation:
    
    I/P
    Z = WX +b
    
    O/P
    A = 1/(1+np.exp(-Z))
    '''
    
    A = 1 / (1 + np.exp(-Z))
    A = A.reshape(Z.shape)
    
    return A

def relu(Z):
    '''
    Function to calculate relu activation:
    
    I/P
    Z = WX +b
    
    O/P
    A = max(0,Z)
    '''
    
    A = np.maximum(0, Z)
    A = A.reshape(Z.shape)
    
    return A

def relugrad(Z):
    '''
    Function to calculate relu gradient:
    
    I/P
    Z = WX +b
    
    O/P
    A = 0 if Z < 0 else 1
    '''
    
    A = Z >= 0
    A = A.reshape(Z.shape)
    
    return A.astype(int)

In [5]:
def linear_function(W,X,b):
    '''
    To calculate Linear function Z
    
    Z = W.T * X + b
    '''

    Z = np.dot(W, X) + b
    Z = Z.reshape(W.shape[0], X.shape[1])
    
    return Z

In [6]:
def cost(A, Y):
    '''
    To calculate cost function for logistice regression
    
    L = -( Y*log(A) + (1-Y)*log(1-A))
    J = - (1/m) * np.sum(Y*log(A) + (1-Y)*log(1-A), keepdims=True, axis=1)
    '''
    
    j = - (1/m) * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A), keepdims=True, axis=1)
    
    return j[0][0]

In [7]:
def forward(W_1,b_1,W_2,b_2,A_0):
    '''
    Function to forward propogate.
    
    O/P
    Predicted value
    '''

    Z_1 = linear_function(W_1,A_0,b_1)
    A_1 = relu(Z_1)
    
    Z_2 = linear_function(W_2,A_1,b_2)
    A_o = sigmoid(Z_2)

    return A_o, A_1, Z_1

In [8]:
def backward(A_o,A_1,A_0,W_2,Z_1,Y):
    '''
    Function to calculate Backward derivatives
    I/P
    A_o, A_1, A_0 = Predicted Probability, Output from Layer 1 and Layer 0
    W_2 = Weight of Layer 2
    Z_1 = Linear function associated with Layer 1
    
    O/P
    returns gradients of W and b
    
    '''
    m = X.shape[1]
    
    dZ_2 = A_o - Y
    dW_2 = -(1/m) * np.dot(dZ_2, A_1.T)
    db_2 = -(1/m) * np.sum(dZ_2, keepdims=True, axis = 1)
    
    dZ_1 = np.dot(W_2.T, dZ_2) * relugrad(Z_1)  
    dW_1 = -(1/m) * np.dot(dZ_1, A_0)
    db_1 = -(1/m) * np.sum(dZ_1, keepdims=True, axis = 1)
    
    return dW_2, db_2, dW_1, db_1

In [9]:
def predict(W_1, b_1, W_2, b_2, X):
    
    X = X.T
    A_o, A_1, Z_1 = forward(W_1, b_1, W_2, b_2, X)
    Y_p = A_o >= 0.5
    
    return Y_p.squeeze().astype(int)

In [10]:
def neural_net(train, test, iterations = 1000, lr=0.01):
    
    # Reshape the train and test parameters
    X = train[0].T
    Y = train[1].reshape(1,-1)
    
    X_test = test[0]
    Y_test = test[1]
    
    n_x = X.shape[0]
    n_h = 20
    W_1, b_1, W_2, b_2 = initialize_weights(n_x, n_h)
    
    Y_p = predict(W_1, b_1, W_2, b_2, X_test)
    test_acc = 1 - np.sum(np.abs(Y_p - Y_test))/Y_test.shape[0]
    print(f'Iteration NA -- NA -- {test_acc}')
    
    for i in range(iterations):
        
        A_o, A_1, Z_1 = forward(W_1,b_1,W_2,b_2,X)
        j = cost(A_o, Y)
        dW_2, db_2, dW_1, db_1 = backward(A_o,A_1,X_train,W_2,Z_1,Y)

        # Update the weights and bias here
        W_1 = W_1 + lr * dW_1
        b_1 = b_1 + lr * db_1
        
        W_2 = W_2 + lr * dW_2
        b_2 = b_2 + lr * db_2
        
        if not i % 100:
            if test:
                Y_p = predict(W_1, b_1, W_2, b_2, X_test)
                test_acc = 1 - np.sum(np.abs(Y_p - Y_test))/Y_test.shape[0]
                print(f'Iteration {i} -- {j} -- {test_acc}')
            
    
    return W_1, b_1, W_2, b_2

In [11]:
W_1, b_1, W_2, b_2 = neural_net( train=(X_train, Y_train), test=(X_test, Y_test), iterations = 1000, lr=0.002)

Iteration NA -- NA -- 0.362
Iteration 0 -- 0.5546992469512698 -- 0.6415
Iteration 100 -- 0.1980516387921557 -- 0.924
Iteration 200 -- 0.18499002892095684 -- 0.9345
Iteration 300 -- 0.18036588893174446 -- 0.938
Iteration 400 -- 0.17746695560702053 -- 0.938
Iteration 500 -- 0.1755357457886904 -- 0.938
Iteration 600 -- 0.17430994112178266 -- 0.9375
Iteration 700 -- 0.173334055778233 -- 0.937
Iteration 800 -- 0.17239511468574134 -- 0.9359999999999999
Iteration 900 -- 0.1733417678987401 -- 0.938
